In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import data

In [4]:
ls Data

calendar.csv                sample_submission.csv
sales_train_validation.csv  sell_prices.csv


In [80]:
calendar=pd.read_csv("Data/calendar.csv")
train=pd.read_csv("Data/sales_train_validation.csv")
prices=pd.read_csv("Data/sell_prices.csv")
sample_submission=pd.read_csv("Data/sample_submission.csv")

#### 1) Calendar

In [19]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
date            1969 non-null object
wm_yr_wk        1969 non-null int64
weekday         1969 non-null object
wday            1969 non-null int64
month           1969 non-null int64
year            1969 non-null int64
d               1969 non-null object
event_name_1    162 non-null object
event_type_1    162 non-null object
event_name_2    5 non-null object
event_type_2    5 non-null object
snap_CA         1969 non-null int64
snap_TX         1969 non-null int64
snap_WI         1969 non-null int64
dtypes: int64(7), object(7)
memory usage: 215.4+ KB


In [22]:
print("Number of day in the", calendar.shape[0])
print("Number of feature for each day", calendar.shape[1])
calendar.head()

Number of day in the 1969
Number of feature for each day 14


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [23]:
calendar.event_name_1.unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [24]:
calendar.event_type_1.unique()

array([nan, 'Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

In [25]:
calendar.event_name_2.unique()

array([nan, 'Easter', 'Cinco De Mayo', 'OrthodoxEaster', "Father's day"],
      dtype=object)

In [26]:
calendar.event_type_2.unique()

array([nan, 'Cultural', 'Religious'], dtype=object)

In [49]:
#What is event_name 2 and event type 2?
#Suspected: 2 events on the same day?

calendar=calendar.fillna(-1) #Fill na
calendar[(calendar.event_name_1!=-1) & (calendar.event_name_2!=-1)]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
85,2011-04-24,11113,Sunday,2,4,2011,d_86,OrthodoxEaster,Religious,Easter,Cultural,0,0,0
827,2013-05-05,11315,Sunday,2,5,2013,d_828,OrthodoxEaster,Religious,Cinco De Mayo,Cultural,1,1,1
1177,2014-04-20,11412,Sunday,2,4,2014,d_1178,Easter,Cultural,OrthodoxEaster,Religious,0,0,0
1233,2014-06-15,11420,Sunday,2,6,2014,d_1234,NBAFinalsEnd,Sporting,Father's day,Cultural,0,1,1
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


Question: Does having the event order switch matter?

In [52]:
# Has event name 1 but not event type 1 ?
print(len(calendar[(calendar.event_name_1!=-1)&(calendar.event_type_1==-1)]))

# Has event name 2 but not event type 2?
print(len(calendar[(calendar.event_name_2!=-1)&(calendar.event_type_2==-1)]))

0
0


### 2) train

In [53]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [59]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [57]:
#Should item_id = length of the dataframe? Answer: Not quite
train.item_id.nunique()

3049

In [62]:
#It seems like each item repeated 10 times....
train.item_id.value_counts().head()

HOUSEHOLD_1_225    10
FOODS_3_720        10
HOUSEHOLD_2_098    10
HOBBIES_2_033      10
HOUSEHOLD_1_369    10
Name: item_id, dtype: int64

In [64]:
#Since there are 10 store in total.
train.store_id.unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [82]:
#Wait, does it mean that all 3049 items are in each store? 
#Answer: Yes, 
day_columns=train.columns[6:]
# train.loc[:,'total_sale'] = train.loc[:,day_columns].sum(axis=1)

#Add all the days by row and check if there is a item =0 
train.loc[:,day_columns].sum(axis=1).sort_values().head()

6682     10
6048     12
27606    13
20192    16
26276    16
dtype: int64

So they are available in all store (appear at least once in 1913 days)!

#### 3) Prices